# 조별과제
1. 2019년 제주도 서귀포시 사고 다발지역 검색하기(XML 파일)
2. 2017년 경기도 시흥시 사고다발지역 검색하기(JSON파일)
3. 2020년 충청북도 청주시 서원구 사고다발지역 검색하기(XML파일)
4. 2018년 서울특별시 동대문구 사고다발지역 이미지 조회
5. 2021년 서울특별시 중랑구 사고다발지역 이미지
   - (가로 1024 x 세로 1024) 조회


> - EndPoint(ServiceURL): `http://apis.data.go.kr/B552061/frequentzoneLg`
>
> - 요청주소(Request URL): `http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg`
>
> - 인증키(encode): `.env` 파일의 `PUBLIC_DATA_SERVICE_KEY` 환경변수 사용

In [ ]:
import os
import requests
from urllib.parse import urlencode, unquote
from pprint import pprint
from xml.etree import ElementTree as ET
from pathlib import Path
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv('../../../.env')

# 공공데이터 포털 인증키를 환경변수에서 로드
SERVICE_KEY_ENCODED = os.getenv("PUBLIC_DATA_SERVICE_KEY", "")
# 요청 시에는 한 번 디코딩해서 넣고, urlencode 가 다시 한 번 인코딩하도록 함
SERVICE_KEY = unquote(SERVICE_KEY_ENCODED)

# 공공데이터포털 문서 기준 요청주소
ENDPOINT = "http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg"

# 응답 결과를 저장할 asset 폴더 설정
ASSET_DIR = Path("asset")
ASSET_DIR.mkdir(exist_ok=True)


def fetch_frequentzone(params: dict, fmt: str = "xml", timeout: int = 5):
    query = {
        "ServiceKey": SERVICE_KEY,              # 문서에 맞춰 대소문자 정렬
        "searchYearCd": params["year"],        # 조회년도 (예: 2019)
        "siDo": params["sido"],                # 시도 코드 (필수)
        "guGun": params["gugun"],              # 시군구 코드 (필수)
        "type": fmt,                            # 응답 형식(xml/json)
        "numOfRows": 100,
        "pageNo": 1,
    }

    url = f"{ENDPOINT}?" + urlencode(query, doseq=True)
    print("요청 URL:", url)

    try:
        res = requests.get(url, timeout=timeout)
    except requests.Timeout:
        print(f"요청 타임아웃 발생 (timeout={timeout}s)")
        return None

    # 4xx / 5xx가 떠도 응답 본문을 확인할 수 있도록 예외 대신 상태/본문을 리턴
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        print("HTTPError 발생:", e)
        print("응답 본문 일부:\n", res.text[:1000])
        # 그대로 리턴해서 호출한 쪽에서 status_code, text 등을 확인 가능
        return res

    return res


# ⚠️ 참고: siDo, guGun 은 반드시 공공데이터 포털 문서에 나온 법정동 코드값을 넣어야 합니다.
# 예시 코드들은 아래 과제 셀에 실제 값으로 채워 두었습니다.

In [ ]:
# 1) 2019년 제주도 서귀포시 사고 다발지역 (XML)
# 법정동 기준: 제주특별자치도(50), 서귀포시(130)
params_jeju_seogwipo_2019 = {
    "year": 2019,
    "sido": "50",   # 제주특별자치도 시도 코드
    "gugun": "130",  # 서귀포시 시군구 코드
}

res_jeju_xml = fetch_frequentzone(params_jeju_seogwipo_2019, fmt="xml")
print("응답 상태 코드:", res_jeju_xml.status_code)
if res_jeju_xml.status_code == 200:
    # XML 앞부분 출력
    print(res_jeju_xml.text[:1000])
    # 파일로 저장
    out_path = ASSET_DIR / "2019_jeju_seogwipo.xml"
    with out_path.open("w", encoding="utf-8") as f:
        f.write(res_jeju_xml.text)
    print(f"저장 완료: {out_path}")
else:
    print(res_jeju_xml.text)  # 에러 메시지 전체 확인

In [ ]:
# 2) 2017년 경기도 시흥시 사고 다발지역 (JSON)
# 법정동 기준: 경기도(41), 시흥시(390)
params_gyeonggi_siheung_2017 = {
    "year": 2017,
    "sido": "41",   # 경기도 시도 코드
    "gugun": "390",  # 시흥시 시군구 코드
}

res_siheung_json = fetch_frequentzone(params_gyeonggi_siheung_2017, fmt="json")
print("응답 상태 코드:", res_siheung_json.status_code)

if res_siheung_json.status_code == 200:
    try:
        data_siheung = res_siheung_json.json()
        pprint(data_siheung)
        # 파일로 저장
        out_path = ASSET_DIR / "2017_gyeonggi_siheung.json"
        with out_path.open("w", encoding="utf-8") as f:
            f.write(res_siheung_json.text)
        print(f"저장 완료: {out_path}")
    except ValueError:
        print("JSON 디코딩 실패:")
        print(res_siheung_json.text[:1000])
else:
    print(res_siheung_json.text[:1000])

In [ ]:
# 3번 2020년 충청북도 청주시 서원구 사고다발지역 검색하기(XML파일)
def fetch_frequentzone(params: dict, fmt: str = "xml", timeout: int = 5):
    query = {
        "ServiceKey": SERVICE_KEY,
        "searchYearCd": params["year"],
        "siDo": params["sido"],
        "guGun": params["gugun"],
        "type": fmt,
        "numOfRows": 100,
        "pageNo": 1,
    }
    url = f"{ENDPOINT}?" + urlencode(query)
    print("요청 URL:", url)
    try:
        res = requests.get(url, timeout=timeout)
        res.raise_for_status()
    except requests.Timeout:
        print(f"타임아웃({timeout}s)")
        return None
    except requests.HTTPError as e:
        print("HTTPError:", e)
        print(res.text[:1000])
        return res
    return res

params_cb_cheongju_seowon_2020 = {"year": 2020, "sido": "43", "gugun": "112"}
res_cb_xml = fetch_frequentzone(params_cb_cheongju_seowon_2020, fmt="xml")
print("응답 상태 코드:", res_cb_xml.status_code)
if res_cb_xml.status_code == 200:
    print(res_cb_xml.text[:1000])
    out_path = ASSET_DIR / "2020_cb_cheongju_seowon.xml"
    with out_path.open("w", encoding="utf-8") as f:
        f.write(res_cb_xml.text)
    print(f"저장 완료: {out_path}")
else:
    print(res_cb_xml.text)

In [ ]:
# 4) 2018년 서울특별시 동대문구 사고 다발지역 이미지 조회 (가로 1024 x 세로 1024)
# 이 API에서 이미지 URL을 직접 주지 않을 수도 있으므로,
# 먼저 JSON/XML 데이터 안에 이미지 경로나 좌표 정보가 있는지 확인해야 합니다.

# 법정동 기준: 서울특별시(11), 동대문구(230)
params_seoul_ddm_2018 = {
    "year": 2018,
    "sido": "11",   # 서울특별시 시도 코드
    "gugun": "230",  # 동대문구 시군구 코드
}

res_ddm = fetch_frequentzone(params_seoul_ddm_2018, fmt="json")
print("응답 상태 코드:", res_ddm.status_code)

if res_ddm.status_code == 200:
    try:
        data_ddm = res_ddm.json()
        pprint(data_ddm)  # 데이터 안에 이미지 관련 필드가 있는지 먼저 확인
        # 파일로 저장
        out_path = ASSET_DIR / "2018_seoul_ddm.json"
        with out_path.open("w", encoding="utf-8") as f:
            f.write(res_ddm.text)
        print(f"저장 완료: {out_path}")
    except ValueError:
        print("JSON 디코딩 실패:")
        print(res_ddm.text[:1000])
else:
    print(res_ddm.text[:1000])

# 만약 이미지 URL 필드가 있다면 아래와 같이 다운로드 예시를 작성할 수 있습니다.
# (필드 이름은 실제 응답 구조에 맞게 수정해야 함)

# from PIL import Image
# from io import BytesIO

# img_url = data_ddm["items"][0]["imgUrl"]  # 예시 키
# img_res = requests.get(img_url)
# img_res.raise_for_status()

# img = Image.open(BytesIO(img_res.content))
# img = img.resize((1024, 1024))
# display(img)

In [ ]:
# 5) 2021년 서울특별시 중랑구 사고 다발지역 이미지 (1024 x 1024)
# 법정동 기준: 서울특별시(11), 중랑구(260)
params_seoul_jungnang_2021 = {
    "year": 2021,
    "sido": "11",   # 서울특별시 시도 코드
    "gugun": "260",  # 중랑구 시군구 코드
}

res_jungnang = fetch_frequentzone(params_seoul_jungnang_2021, fmt="json")
print("응답 상태 코드:", res_jungnang.status_code)

if res_jungnang.status_code == 200:
    try:
        data_jungnang = res_jungnang.json()
        pprint(data_jungnang)
        # 파일로 저장
        out_path = ASSET_DIR / "2021_seoul_jungnang.json"
        with out_path.open("w", encoding="utf-8") as f:
            f.write(res_jungnang.text)
        print(f"저장 완료: {out_path}")
    except ValueError:
        print("JSON 디코딩 실패:")
        print(res_jungnang.text[:1000])
else:
    print(res_jungnang.text[:1000])



In [ ]:
# 6) 2021년 서울특별시 중랑구 사고다발지역 이미지 (1024 x 1024) 생성 및 저장
import json
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["font.family"] = "NanumGothic"
mpl.rcParams["axes.unicode_minus"] = False

jungnang_path = ASSET_DIR / "2021_seoul_jungnang.json"
with jungnang_path.open(encoding="utf-8") as f:
    jungnang_data = json.load(f)

items = jungnang_data["items"]["item"]


fig, ax = plt.subplots(figsize=(10.24, 10.24), dpi=100)  

for item in items:
    geom = json.loads(item["geom_json"])  
    coords = geom["coordinates"][0]       
    xs = [p[0] for p in coords]           
    ys = [p[1] for p in coords]           

    ax.fill(xs, ys, alpha=0.4, label=item["spot_nm"]) 
    ax.plot(xs, ys, color="black", linewidth=1)       

ax.set_aspect("equal", "box")
ax.set_xlabel("경도")
ax.set_ylabel("위도")
ax.set_title("2021년 서울특별시 중랑구 사고다발지역")
ax.legend(fontsize=6, loc="upper right")

out_img_path = ASSET_DIR / "2021_seoul_jungnang.png"
plt.savefig(out_img_path, dpi=100, bbox_inches="tight")
plt.show()
plt.close(fig)
plt.show()


In [ ]:
import requests
import json
url = "http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg"
params = {
  "ServiceKey" : SERVICE_KEY,
  "searchYearCd" : "2017",
  "siDo" : "41",
  "guGun" : "390",
  "numOfRows" : "100",
  "pageNo" : "1",
  "type" : "json"
}
response = requests.get(url, params=params)
sh_list = json.loads(response.text)
sh_list['items']['item'][0]['spot_nm']
n = (len(sh_list['items']['item'][0]))
n
# n = len(sh_list['items']['item'])
# spot_list = []
# for i in range(n):
#     temp = sh_list['items']['item'][i]['spot_nm']
#     spot_list.append(temp)
    
# print(spot_list)


In [ ]:

import requests
import json
url = "http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg"
params = {
  "ServiceKey" : SERVICE_KEY,
  "searchYearCd" : "2017",
  "siDo" : "41",
  "guGun" : "390",
  "numOfRows" : "100",
  "pageNo" : "1",
  "type" : "json"
}
response = requests.get(url, params=params)
# print(response.url)
# print(response.status_code)
# print(response.text)
sh_list = json.loads(response.text)
print(sh_list)
#sh_list['items']['item'][0]['spot_nm']
n = len(sh_list['items']['item'])
spot_list = []
for i in range(n):
    temp = sh_list['items']['item'][i]['spot_nm']
    spot_list.append(temp)
    
print(spot_list)